In [53]:
# Bee Hive data https://drive.google.com/file/d/142IBcs6OyQiJxO7owPfkEBFbkrudnh0g/view?usp=sharing

In [54]:
APP = 'BeeHive'

In [55]:
# Install a pip package in the current Jupyter kernel
#! '../../package_py.bash'

In [56]:
from pyspark.sql import SparkSession,Row
from pyspark.sql.types import StructType, IntegerType, StringType
from pyspark.sql.functions import split, row_number, udf, col, min, when
from pyspark.sql.window import Window
from pyspark.shell import sqlContext

from pep_data.project import quick_conf
from src.pep_data.spark.util import base_df

import random

In [57]:
# Create Spark session
spark = SparkSession.builder.appName(APP).getOrCreate()

In [58]:
# Get app configuration from project.conf file
conf = quick_conf()

In [59]:
df = base_df(spark, conf, APP)

df.show()

+------+------------------------+-----------+-----------+------------+-----------+-----------+
|Bee ID|DaughtersEfficiencyScore|Father SIZE|Father TYPE|           X|          Y|          Z|
+------+------------------------+-----------+-----------+------------+-----------+-----------+
|   0_0|           -0.1647494899|          5|     107027|    5.345708|25.54008605|24.79858692|
|   0_9|          -0.09111780618|          8|      35473|  3.61617713|15.39993678|14.11150683|
|  0_16|           -0.2783737487|          9|      72732| 6.487132473|27.96111467|23.50405554|
|  0_76|           0.01988315069|          6|      49069| 8.285176906|21.88111447|10.80561155|
|  0_35|           -0.0758420403|          5|        187|        -1.0|       13.0|       17.0|
|  0_17|           -0.3362110457|          5|      74276|        -0.6|       12.0|       16.0|
|  0_49|           -0.1311013747|         10|     108253| 4.123056465|16.28346035| 12.7359525|
|  0_11|           -0.1881197044|          5|     

In [60]:
# Split Bee_ID column to Cycle(Bee value) and Cycle ID(ID value) columns
df_cleaned = df.withColumn('Cycle', split(col('Bee ID'), '_')\
                           .getItem(0))\
               .withColumn('Cycle ID', split(col('Bee ID'), '_')\
                           .getItem(1))

# Change the type of value in Cycle column from string to integer
df_cleaned= df_cleaned.withColumn("Cycle",col("Cycle")\
                                  .cast(IntegerType()))
df_cleaned.show()

+------+------------------------+-----------+-----------+------------+-----------+-----------+-----+--------+
|Bee ID|DaughtersEfficiencyScore|Father SIZE|Father TYPE|           X|          Y|          Z|Cycle|Cycle ID|
+------+------------------------+-----------+-----------+------------+-----------+-----------+-----+--------+
|   0_0|           -0.1647494899|          5|     107027|    5.345708|25.54008605|24.79858692|    0|       0|
|   0_9|          -0.09111780618|          8|      35473|  3.61617713|15.39993678|14.11150683|    0|       9|
|  0_16|           -0.2783737487|          9|      72732| 6.487132473|27.96111467|23.50405554|    0|      16|
|  0_76|           0.01988315069|          6|      49069| 8.285176906|21.88111447|10.80561155|    0|      76|
|  0_35|           -0.0758420403|          5|        187|        -1.0|       13.0|       17.0|    0|      35|
|  0_17|           -0.3362110457|          5|      74276|        -0.6|       12.0|       16.0|    0|      17|
|  0_49|  

In [61]:
# Sort the data frame by Cycle column and add row number for each row(new column with the name Continuous ID)
w = Window().orderBy('Cycle')
df_cleaned = df_cleaned.withColumn('Continuous ID', row_number()\
                                   .over(w))

df_cleaned.show()

+------+------------------------+-----------+-----------+------------+-----------+-----------+-----+--------+-------------+
|Bee ID|DaughtersEfficiencyScore|Father SIZE|Father TYPE|           X|          Y|          Z|Cycle|Cycle ID|Continuous ID|
+------+------------------------+-----------+-----------+------------+-----------+-----------+-----+--------+-------------+
|   0_0|           -0.1647494899|          5|     107027|    5.345708|25.54008605|24.79858692|    0|       0|            1|
|   0_9|          -0.09111780618|          8|      35473|  3.61617713|15.39993678|14.11150683|    0|       9|            2|
|  0_16|           -0.2783737487|          9|      72732| 6.487132473|27.96111467|23.50405554|    0|      16|            3|
|  0_76|           0.01988315069|          6|      49069| 8.285176906|21.88111447|10.80561155|    0|      76|            4|
|  0_35|           -0.0758420403|          5|        187|        -1.0|       13.0|       17.0|    0|      35|            5|
|  0_17|

In [62]:
# Create a dictionary with key = cycle , value = minimum value of Continuous ID of cycle(key)
continuous_min_id_per_cycle = {key : value for key, value  in df_cleaned.groupBy('Cycle').min('Continuous ID').collect()}

# continuous_min_id_per_cycle

In [63]:
# Create sorted list of all distinct values of Cycle column
cycles = sorted([i[0] for i in df_cleaned.select('Cycle').distinct().collect()])

# cycles

In [64]:
# Return parent continuous id according to cycle and n
def assert_parent_bee_id(cycle):
    n = 3

    # Get index  of cycle in cycles list
    cycle_index = cycles.index(cycle)

    # Return parent bee id if cycle is 0
    if  not cycle_index :
        return None

    min_cycle_index = 0

    # Update min_cycle_index according to cycle_index and n
    if cycle_index > n:
        min_cycle_index = cycle_index - n

    # Calculate the minimum value for random parent continuous id
    min_rand_value = continuous_min_id_per_cycle[cycles[min_cycle_index]]

    # Calculate the maximum value for random parent continuous id
    max_rand_vale = continuous_min_id_per_cycle[cycles[cycle_index]] - 1

    # Get random value of parent_continuous_id (from min_rand_value to max_rand_vale)
    parent_continuous_id = random.randint(min_rand_value, max_rand_vale)

    return parent_continuous_id

In [65]:
# TODO group by some columns to create partitions

# Convert assert_parent_bee_id(cycle) to user defined function
assert_parent_bee_id_udf = udf(lambda z: assert_parent_bee_id(z))

# Create new column Parent Continuous ID using the assert_parent_bee_id_udf function and Cycle column
# cache() caches the specified data frame in the memory of your cluster's workers
# If executing multiple actions on the same data frame then cache it
df_cleaned = df_cleaned.withColumn("Parent Continuous ID", assert_parent_bee_id_udf(col('Cycle')))\
                        .cache()

#df_cleaned.show()

In [66]:
# Create a DataFrame of parent bees
df_parent_bees = df_cleaned.select('Bee ID', 'Continuous ID')
df_parent_bees = df_parent_bees.withColumnRenamed("Bee ID","Parent Bee ID")
df_parent_bees = df_parent_bees.withColumnRenamed("Continuous ID","Temp Continuous ID")
df_parent_bees= df_parent_bees.withColumn("Temp Continuous ID",col("Temp Continuous ID")\
                                  .cast(StringType()))

#df_parent_bees.show()

In [67]:
# Add the parent bee id to each bee id with the use of join
df_beeId_parent_beeId= df_cleaned.join(df_parent_bees, df_cleaned['Parent Continuous ID'] == df_parent_bees['Temp Continuous ID'],'left').drop('Temp Continuous ID')
#df_beeId_parent_beeId.show()

In [68]:
# Split Bee_ID column to Cycle(Bee value) and Cycle ID(ID value) columns
df_beeId_parent_beeId = df_beeId_parent_beeId.withColumn('Parent Cycle', split(col('Parent Bee ID'), '_')\
                           .getItem(0))


# Change the type of value in Cycle column from string to integer
df_beeId_parent_beeId= df_beeId_parent_beeId.withColumn("Parent Cycle",col("Parent Cycle")\
                                  .cast(IntegerType()))
# df_beeId_parent_beeId.show()

In [69]:
# Add oldest Ancestor column
df_beeId_parent_beeId = df_beeId_parent_beeId.withColumn('Oldest Ancestor',
                                                         when(col('Cycle')==0,None).
                                                         otherwise(''))
# df_beeId_parent_beeId.toPandas()

In [70]:
# Add Ancestors column
df_beeId_parent_beeId = df_beeId_parent_beeId.withColumn('Ancestors',
                                                         when(col('Cycle')==0,None).
                                                         otherwise(''))
# df_beeId_parent_beeId.toPandas()

In [71]:
# Assert the values of Ancestors and old Oldest Ancestor colums for the children of cycle 0
def assert_ancestors_and_oldestancestors_columns_children(row, list_cycle_0):
    row_dict = row.asDict()

    for bee in list_cycle_0:
        if row_dict['Parent Continuous ID'] == str(bee['Continuous ID']):
            row_dict['Oldest Ancestor'] = bee['Bee ID']+'-'+str(bee['Continuous ID'])
            row_dict['Ancestors'] = bee['Bee ID']+'-'+str(bee['Continuous ID'])
            break

    newrow = Row(**row_dict)

    return newrow

In [72]:
# Data frame with additional columns with only cycle 0
df_enriched = df_beeId_parent_beeId.filter(col('Cycle')==0)
# Data frame with only cycle 0 children
df_parent_cycle = df_beeId_parent_beeId.filter(col('Parent Cycle')==0)

list_cycle_0 = df_enriched.collect()
# Convert the data frame to rdd and assert the values of Oldest Ancestor and Ancestors columns
df_parent_cycle_rdd = df_parent_cycle.rdd.map(lambda row: assert_ancestors_and_oldestancestors_columns_children(row,list_cycle_0)).cache()
df_parent_cycle = sqlContext.createDataFrame(df_parent_cycle_rdd,schema= df_enriched.schema)
# Union the initial enriched data frame with the result
df_enriched = df_enriched.union(df_parent_cycle)
df_enriched.toPandas()

Py4JJavaError: An error occurred while calling o849.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 45.0 failed 1 times, most recent failure: Lost task 0.0 in stage 45.0 (TID 55) (DESKTOP-2H598RF.mshome.net executor driver): java.io.IOException: Cannot run program "python3": CreateProcess error=3, The system cannot find the path specified
	at java.lang.ProcessBuilder.start(ProcessBuilder.java:1048)
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:167)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:164)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:81)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.s

In [ ]:
# Assert the values of Ancestors and old Oldest Ancestor colums for the children of cycle n (all cycles except cycle 0)
def assert_ancestors_and_oldestancestors_columns_grandchildren(row, list_n_cycles):

    row_dict = row.asDict()

    for bee in list_n_cycles:
        if row_dict['Parent Continuous ID'] == str(bee['Continuous ID']):
            row_dict['Oldest Ancestor'] = bee['Oldest Ancestor']
            row_dict['Ancestors'] = bee['Ancestors'] + '->' +row_dict['Parent Bee ID']+'-'+str(row_dict['Parent Continuous ID'])
            break

    newrow = Row(**row_dict)

    return newrow

In [ ]:
# For all the cycle except the first, get df of children pf cycle c, get list of its parents
# convert df to rdd and use the assert_ancestors_and_oldestancestors_columns_grandchildren to assert
# values to the new columns, union the result with the existing enriched df
n=3
for cycle in cycles[1:-1:]:
    df_cycle = df_beeId_parent_beeId.filter(col('Parent Cycle')==cycle)
    # list_n_cycles = df_enriched.select('Cycle', 'Continuous ID', 'Oldest Ancestor','Ancestors' )\
    #                             .filter(col('Cycle')==cycle).collect()
    list_n_cycles = df_enriched.select('Cycle', 'Continuous ID', 'Oldest Ancestor','Ancestors' )\
                                .filter((col('Parent Cycle')<cycle) & (col('Parent Cycle')>=cycle-n)).collect()

    # Convert the data frame to rdd and assert the values of Oldest Ancestor and Ancestors columns
    df_cycle_rdd = df_cycle.rdd.map(lambda row: assert_ancestors_and_oldestancestors_columns_grandchildren(row,list_n_cycles)).cache()
    df_cycle = sqlContext.createDataFrame(df_cycle_rdd,schema= df_enriched.schema)
    # Union the enriched data frame with the result
    df_enriched = df_enriched.union(df_cycle)

In [ ]:
# Save data frame to csv file
file_name = '2.csv'
save_path = conf[APP]['data_save']
df_enriched.coalesce(1).write.option('header',True).\
csv(save_path+file_name)

In [ ]:
from pyspark.sql.functions import collect_list, collect_set
df_cycle_beeID_list = df_beeId_parent_beeId.groupBy('Cycle').agg(collect_list('Bee ID'))
df_cycle_beeID_list.toPandas()
#df_cycle_beeID_list.foreach()